<a href="https://colab.research.google.com/github/EvagAIML/014-NLP-Model-v1/blob/main/Copy_of_Medical_Diagnostic_Tool_v16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROTOTYPE
## Medical Diagnostic Assistant: AI-Powered Question Answering



--

## Executive Summary

This initiative delivers a clinically focused medical diagnostic support system designed to improve the reliability of AI-generated medical information. The solution employs a Retrieval-Augmented Generation (RAG) architecture that constrains all model outputs to content retrieved from a single authoritative reference source, The Merck Manual of Diagnosis & Therapy. By grounding responses in cited source material, the system addresses a key limitation of large language models in healthcare contexts: the risk of producing fluent but unverifiable or unsupported medical guidance.

The current implementation demonstrates measurable value. Compared to an ungrounded baseline language model, the RAG-based system consistently produces answers that are verifiable, traceable to source pages, and aligned with accepted clinical standards. Automated evaluation using a structured LLM-as-a-Judge framework reports average relevance and groundedness scores of approximately 4.8 out of 5. Collectively, these results indicate that the system can reduce information retrieval time, promote consistency in diagnostic reasoning, and serve as a reliable clinical decision-support aid rather than an unconstrained generative tool.

## Problem Statement

### Business Context
The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

### Objective
As an AI specialist, the task is to develop a **Retrieval-Augmented Generation (RAG)** solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

**Common Questions to Answer**

1. **Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"
2. **Surgical Inquiry**: "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
3. **Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"
4. **Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"
5. **Dermatology**: "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
6. **Neurology**: "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
7. **Emergency Care**: "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
8. **Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"
9. **Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Data Description
The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs.

The manual is provided as a PDF: `014-NLP-PROJ-medical_diagnosis_manual_19.pdf`

## Environment Initialization

### Environment Setup and Dependency Configuration

**Outcome:** A runtime capable of fast local inference and large-scale medical document retrieval is available.

**Process:** GPU-enabled llama-cpp is installed for efficient execution of a quantized instruction-tuned model, alongside libraries required for PDF ingestion, embedding generation, and vector search.

In [ ]:
# ============================================================
# Installation for GPU llama-cpp-python
# ============================================================
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q 2>/dev/null

# Installing other dependencies
!pip install huggingface_hub==0.35.3 pandas==2.2.2 tiktoken==0.12.0 pymupdf==1.26.5 langchain==0.3.27 langchain-community==0.3.31 chromadb==1.1.1 sentence-transformers==5.1.1 numpy==1.26.4 -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 99.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 265.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 291.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 311.5 MB/s eta 0:00:00


**Note:** Please restart the runtime after installation.

### Core Imports and Shared Configuration

**Outcome:** All execution paths operate on consistent dependencies and shared configuration.

**Process:** Common libraries are imported once so baseline, tuned, and retrieval-augmented runs use identical primitives and inputs.

In [ ]:
# ============================================================
# Core Imports and Shared Configuration
# ============================================================

# Dependencies
import os
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Global Qs
questions = [
    "What are the common symptoms and treatments for pulmonary embolism?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Can you provide the trade names of medications used for treating hypertension?",
    "What are the first-line options and alternatives for managing rheumatoid arthritis?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",
    "What are the diagnostic steps for suspected endocrine disorders?",
    "What is the protocol for managing sepsis in a critical care unit?"
]

## Baseline Medical Question Answering (Ungrounded)

### Baseline Model Initialization

**Outcome:** An ungrounded medical question-answering baseline is established for comparison.

**Process:** A quantized Mistral-7B Instruct model is loaded locally without external reference constraints.

In [ ]:
# ============================================================
# Load and Initialize LLM
# ============================================================


# Load LLM
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

# Initialize LLM
llm = Llama(
    model_path=model_path,
    n_ctx=4096,        # Context window
    n_gpu_layers=-1,   # Offload all to GPU suitable for T4
    n_batch=512,
    verbose=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


###Inference Wrapper

**Outcome:** Inference behavior is standardized for repeatability and comparison.

**Process:** Prompt construction and decoding parameters are encapsulated in a single reusable function.

In [ ]:
# ============================================================
# Inference Wrapper
# ============================================================

def generate_response(query, max_tokens=2048, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

### Baseline Question Execution

**Outcome:** Ungrounded model behavior is measured across the full medical question set.

**Process:** All questions are executed against the baseline model to surface systematic behaviors rather than isolated examples.

In [ ]:
# ============================================================
# Baseline Question Execution
# ============================================================

vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?
A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries supplying the lungs with blood. This can lead to decreased oxygen supply to the body, which can be life-threatening. Here are some common symptoms of pulmonary embolism:

1. Shortness of breath: This is often the first and most prominent symptom of PE. The person may feel like they're unable to get enough air or that they're breathing too hard for the level of activity they're engaged in.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity. The pain may be described as feeling like a heavy weight on the chest.
3. Rapid heart rate: PE can cause the heart to beat faster than normal in an attempt to pump more blood through the body and deliver oxygen to the lungs.
4. Coughing up b

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain usually starts around the navel area and then shifts to the lower right side of the abdomen. It may be constant or come and go, and it can range from mild to severe.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially if the inflammation is severe.
4. Fever: A fever above 101 degrees Fahrenheit (38.3 degrees Celsius) is a common sign of appendicitis.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others may have constipation.
6. Abdominal swelling: The abdomen may become swollen and tender to the touch.
7. Inability to pass gas: Passing gas can provide relief from abdominal pain, but if 

Llama.generate: prefix-match hit


A: Yes, I can provide a list of some common trade names of medications used for treating hypertension (high blood pressure). Please note that this is not an exhaustive list and there are many other brand names available depending on the specific class of medication and formulation.

1. Thiazide diuretics: Diuril (chlorothiazide), Maxzide (triamterene/hydrochlorothiazide), Dyazide (hydrochlorothiazide/triamterene)
2. Beta-blockers: Tenormin (atenolol), Lopressor (metoprolol), Toprol XL (metoprolol succinate), Bisoprolol, Labetalol
3. ACE inhibitors: Zestril (lisinopril), Monopress (fosinopril), Vasotec (enalapril), Lotensin (benazepril)
4. ARBs (Angiotensin II Receptor Blockers): Cozaar (losartan), Valsartan, Avapro (irbesartan), Candesartan
5. Calcium channel blockers: Norvasc (amlodipine), Procardia XL (nifedipine extended-release), Diltia XT (diltiazem extended-release)
6. Diuretics - Potassium-sparing: Aldactone (spironolactone), Microzide (hydrochlorothiazide/triamterene)
7. Minera

...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the person's condition. If the fracture is open or the person is in severe pain, shock, or unable to move, do not attempt to move them without professional help. Call for emergency medical assistance immediately.

2. Provide first aid: If the person is stable and able to be moved, provide first aid by immobilizing the leg with a splint or sling. Use a makeshift splint if necessary, such as a stick or branch, secured with clothing or other materials. Apply a sterile dressing to any open wounds and apply pressure to control bleeding.

3. Transportation: If the person is unable to walk, use a stretcher or carry them out of the wilderness area safely. Make sure they are kept warm and comfortable during transportation.

4. Medi

Llama.generate: prefix-match hit


A: 1. Thorough Clinical Evaluation: A comprehensive clinical evaluation is essential to identify any symptoms suggestive of an endocrine disorder. This may include assessing medical history, family history, and performing a physical examination.

2. Laboratory Tests: Various laboratory tests are used to diagnose endocrine disorders. These tests may include:
   - Hormone level measurements: Measuring the levels of specific hormones in the blood or urine can help identify an endocrine disorder. For example, measuring insulin levels for diabetes mellitus or thyroid stimulating hormone (TSH) and free thyroxine (FT4) levels for thyroid disorders.
   - Glucose tolerance test: This test measures the body's response to glucose intake and can help diagnose diabetes mellitus.
   - Imaging studies: Imaging tests such as ultrasound, CT scan, or MRI may be used to visualize glands and organs involved in endocrine function.
   - Genetic testing: In some cases, genetic testing may be necessary to ide

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition and diagnosis: Identify sepsis early by recognizing signs and symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, and lactic acidosis. Use validated scoring systems like the Sequential Organ Failure Assessment (SOFA) score or the Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA) to help identify patients at risk for sepsis.
2. Resuscitation: Begin resuscitation as soon as possible, focusing on restoring intravascular volume and maintaining adequate tissue perfusion. This may involve administering fluids, vasopressors, and/or inotropes.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on the suspected source of infection. Consider 

### Baseline Analysis

Ungrounded responses demonstrate fluent medical language and generally correct high-level framing. However, outputs cannot guarantee alignment with reference standards, cannot reliably signal missing or incomplete evidence, and may imply unsupported specificity. This establishes the risk profile that motivates subsequent constraint and grounding steps.

## Prompt Engineering and Decoding Control (Parameter Tuning)

### Prompt Configuration Evaluation

**Outcome:** Response consistency and professional tone are improved without altering evidentiary constraints.

**Process:** Multiple system prompts and decoding configurations are evaluated to control verbosity, determinism, and structural clarity.

In [ ]:
# ============================================================
# Prompt Configuration Evaluation
# ============================================================

# Define 5 Configurations

configs = [
    {"name": "High Creativity", "temp": 0.9, "sys": "You are a helpful medical assistant.", "top_k": 100},
    {"name": "Strict Professional", "temp": 0.0, "sys": "You are a concise, professional doctor. Answer only what is asked.", "top_k": 10},
    {"name": "Chain of Thought", "temp": 0.3, "sys": "Think step-by-step. Explain the reasoning before giving the final answer.", "top_k": 40},
    {"name": "ELI5", "temp": 0.7, "sys": "Explain like I am 5 years old.", "top_k": 50},
    {"name": "Balanced", "temp": 0.4, "sys": "You are a knowledgeable assistant. Provide detailed clinical information.", "top_k": 50}
]

# We will test these on all questions
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== Testing Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in configs:
        print(f"--- {cfg['name']} ---")
        ans = generate_response(
            question,
            temperature=cfg['temp'],
            top_k=cfg['top_k'],
            system_prompt=cfg['sys']
        )
        print(f"{ans}\n")


=== Testing Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- High Creativity ---


Llama.generate: prefix-match hit


I'm glad you asked about pulmonary embolism as it's an important condition to be aware of. A pulmonary embolism occurs when a blood clot or other foreign substance travels to the lungs, blocking or reducing blood flow in the arteries supplying the lungs. Here are some common symptoms and treatments for this condition:

Symptoms:
1. Shortness of breath: This is one of the most common symptoms of pulmonary embolism. You may feel like you can't catch your breath or that you're breathing very hard, even when at rest.
2. Chest pain: Pulmonary embolism can cause sharp chest pain, particularly when taking deep breaths or coughing. The pain may worsen with movement or exertion.
3. Rapid heart rate: Your heart may beat faster than usual in response to the reduced blood flow to your lungs.
4. Cough: A persistent cough can be a sign of pulmonary embolism, especially if it produces bloody phlegm.
5. Swelling: You may experience swelling in your legs, ankles, or arms due to the buildup of fluid cau

Llama.generate: prefix-match hit


A pulmonary embolism (PE) is a blockage in the arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Common symptoms include:

1. Shortness of breath
2. Chest pain or discomfort, often worsening with deep breathing or coughing
3. Rapid heart rate
4. Anxiety
5. Coughing up blood
6. Swelling in the legs or arms
7. Fainting

Treatment for PE depends on the severity of the blockage and the overall health of the patient. Options include:

1. Anticoagulant medications to prevent clot formation and growth
2. Thrombolytic therapy, which involves administering drugs that dissolve blood clots
3. Embolectomy, a surgical procedure to remove the clot from the lung artery
4. Supportive measures such as oxygen therapy, fluid replacement, and pain management.

It's important to note that PE can be life-threatening if not treated promptly, so seeking medical attention right away is crucial.

--- Chain of Thought ---


...

Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a systematic approach. Here are the key steps in the sepsis management protocol:

1. Early recognition: Identify sepsis suspects based on clinical suspicion, using tools like the Quick Sequential Organ Failure Assessment (qSOFA) or the Systemic Inflammatory Response Syndrome (SIRS) criteria.

2. Rapid resuscitation: Initiate fluid resuscitation with intravenous crystalloids to maintain adequate tissue perfusion and organ function. The goal is to achieve a mean arterial pressure of 65 mmHg or higher and a central venous oxygen saturation (ScvO2) of 70% or above.

3. Antibiotics administration: Start broad-spectrum antibiotics as soon as possible, ideally within the first hour of recognition, to cover suspected pathogens while awaiting culture results.

4. Source control: Identify and address the source of infection, which may include surgical

Llama.generate: prefix-match hit


Sepsis is when your body gets really sick from an infection, and we need to help it get better quickly in the hospital's special care unit called the Critical Care Unit (CCU). Here's what the doctors and nurses do:

1. Give medicines to fight the infection: The doctors give strong medicines through an IV line to kill the germs that are making you sick.

2. Keep you hydrated: The nurses give you fluids through your IV to help keep you hydrated, as you might not be able to drink or eat much.

3. Give you oxygen: If you have trouble breathing, they will give you oxygen through a mask to help you breathe easier.

4. Monitor your vitals: The nurses check your heart rate, blood pressure, temperature, and oxygen levels frequently to ensure that everything is working correctly.

5. Provide nutrition: They may also provide you with nutrients through the IV or a feeding tube to help your body recover.

6. Support your organs: If your kidneys or other organs are not functioning properly, the doct

Llama.generate: prefix-match hit


Sepsis is a life-threatening condition caused by the body's response to an infection, which can lead to tissue damage and organ failure. The management of sepsis in a critical care unit involves a multidisciplinary approach that focuses on early recognition, rapid resuscitation, source control, and supportive care. Here is a detailed protocol for managing sepsis in a critical care unit:

1. Early Recognition:
- Identify patients at risk of developing sepsis based on clinical suspicion, laboratory results, or the presence of infection.
- Use validated scoring systems such as Sequential Organ Failure Assessment (SOFA) score, Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA), or the National Early Warning Score (NEWS) to identify patients who may be developing sepsis.

2. Rapid Resuscitation:
- Initiate fluid resuscitation with isotonic crystalloids to maintain adequate tissue perfusion and organ function.
- Monitor hemodynamic parameters such as mean arterial pressure, c

### Prompting Analysis

Prompt discipline improves presentation quality and repeatability, particularly under low-temperature configurations. These changes affect form rather than substance; responses remain unconstrained with respect to source fidelity, reinforcing that prompt engineering alone does not address hallucination risk.

## Reference Preparation for Retrieval-Augmented Generation

### Reference Document Ingestion

**Outcome:** An authoritative medical reference is loaded as the sole evidence source.

**Process:** The Merck Manual PDF is ingested and validated for downstream processing.

In [ ]:
# ============================================================
# Reference Document Ingestion
# ============================================================

# Dependencies
import requests
import os
from langchain_community.document_loaders import PyMuPDFLoader

doc_path = "014-NLP-PROJ-medical_diagnosis_manual_19.pdf"
github_url = "https://github.com/EvagAIML/014-NLP-Model-v1/blob/main/014-NLP-PROJ-medical_diagnosis_manual_19.pdf?raw=true"

# Download the PDF file from GitHub
if not os.path.exists(doc_path):
    print(f"Downloading {doc_path} from GitHub...")
    try:
        response = requests.get(github_url, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        with open(doc_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from GitHub: {e}")
        print("Please ensure the URL is correct and accessible.")

if os.path.exists(doc_path):
    loader = PyMuPDFLoader(doc_path)
    docs = loader.load()
    print(f"Loaded {len(docs)} pages.")

    # Check first 5 pages
    for i in range(min(5, len(docs))):
        print(f"--- Page {i+1} ---")
        print(docs[i].page_content[:500])
        print("...")
else:
    print(f"File {doc_path} not found even after attempted download. Please ensure it is in the working directory or the download path is correct.")

Download complete.
Loaded 4114 pages.
--- Page 1 ---
erikvdesigner@gmail.com
U36PAIRLB4
ant for personal use by erikvdesigner@g
shing the contents in part or full is liable
...
--- Page 2 ---
erikvdesigner@gmail.com
U36PAIRLB4
This file is meant for personal use by erikvdesigner@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
...
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .................................
...
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    ............................................

### Reference Text Chunking

**Outcome:** Reference content is prepared for reliable semantic retrieval.

**Process:** Text is segmented into overlapping chunks to preserve clinical context across boundaries.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)
print(f"Total Chunks created: {len(chunks)}")

Total Chunks created: 18020


### Vector Index Construction

**Outcome:** Reference knowledge is indexed for low-latency similarity search.

**Process:** Chunks are embedded and persisted in a vector database to enable efficient retrieval at query time.

In [ ]:
# ============================================================
# Embedding & Vector Store
# ============================================================

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db_medical_final"
)

# Default Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

/tmp/ipython-input-4154461508.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Retrieval Preparation Analysis

Retrieval quality becomes the dominant factor in answer reliability. Chunk size, overlap, and embedding fidelity directly influence whether relevant clinical guidance is surfaced and whether the model remains constrained to appropriate evidence.

## Retrieval-Augmented Medical Question Answering

### RAG Answer Generation

**Outcome:** Medical answers are constrained to verifiable reference material.

**Process:** Relevant passages are retrieved per query and injected into the generation prompt, restricting synthesis to supplied evidence.

In [ ]:
# ============================================================
# RAG Generation Function
# ============================================================

def generate_rag_answer(query, k=3, temperature=0.1, max_tokens=2048):
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query, k=k)

    # Format context and extract sources
    context_list = []
    sources_info = []
    for d in retrieved_docs:
        page_num = d.metadata.get('page', 'Unknown')
        # Assuming page is 0-indexed integer from PyMuPDF
        if isinstance(page_num, int):
            page_num += 1
        src_str = f"Page {page_num}"
        sources_info.append(src_str)
        context_list.append(f"[{src_str}] {d.page_content}")

    context = "\n\n".join(context_list)
    unique_sources = ", ".join(sorted(list(set(sources_info))))

    # Construct Prompt
    sys_msg = "You are a professional medical assistant for healthcare professionals. Use the following Context to answer the Question. If the answer is not in the context, say so. Do not advise consulting a healthcare professional."
    user_msg = f"Context:\n{context}\n\nQuestion: {query}\n\nPlease cite specific Page numbers and Sections from the text in your answer."

    prompt = f"[INST] {sys_msg}\n{user_msg} [/INST]"

    # Generate
    output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=["[/INST]"]
    )
    return output['choices'][0]['text'].strip(), context

In [ ]:
# ============================================================
# RAG Parameter Tuning (5 Combinations)
# ============================================================

rag_configs = [
    {"name": "C1: Low Context, Precise", "k": 2, "temp": 0.0},
    {"name": "C2: Standard", "k": 3, "temp": 0.1},
    {"name": "C3: High Context", "k": 5, "temp": 0.1},
    {"name": "C4: Creative", "k": 3, "temp": 0.7},
    {"name": "C5: Max Context, Strict", "k": 7, "temp": 0.0}
]

print("Running RAG Tuning on all questions")
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== RAG Tuning Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in rag_configs:
        print(f"--- {cfg['name']} ---")
        ans, _ = generate_rag_answer(question, k=cfg['k'], temperature=cfg['temp'])
        print(f"A: {ans}\n")

Running RAG Tuning on all questions

=== RAG Tuning Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- C1: Low Context, Precise ---


Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081 of Chapter 194, the common symptoms for pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, syncope or cardiorespiratory arrest in severe cases, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound.

The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and the insertion of an inferior vena caval filter (Section: "Treatment" on page 2080).

Therefore, the answer is: The common symptoms for pulmonary embolism are dyspnea, pleuritic chest pain, cough, syncope or cardiorespiratory arrest in severe cases, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound (Page 2080). The treatment includes anticoagulants, thrombo

Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081, the common symptoms of pulmonary embolism include dyspnea, pleuritic chest pain, cough, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound. (Section: Introduction)

The treatment for pulmonary embolism involves anticoagulants and, in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and inferior vena caval filters. (Section: Treatment)

Please note that the context also mentions other conditions such as pneumonia, sepsis, foreign body embolism, tumor embolism, pulmonary-renal syndromes, and chronic illnesses that may present with similar symptoms or signs. It is essential to consider these possibilities in the diagnostic evaluation. (Sections: signs of pneumonia or sepsis, Foreign body embolism, Tumor embolism, Several specific findings are of note)

--- C3: High Context ---


...

A: According to the context provided, the protocol for managing sepsis in a critical care unit involves several steps. First, supportive care includes keeping the patient warm, controlling hemorrhage, checking and assisting with airway and ventilation, and providing supplemental oxygen (Page 2401). If the patient is experiencing septic shock, treatment begins simultaneously with evaluation and includes providing supplemental oxygen by face mask, intubation with mechanical ventilation if necessary, and insertion of two large IV catheters or a central venous line or intraosseous needle (Page 2454). General management also involves aggressive fluid resuscitation, antibiotics, surgical excision of infected or necrotic tissues, drainage of pus, and supportive care. The treatment may also include intensive control of blood glucose and administration of corticosteroids and activated protein C (Page 2454).

Symptoms of sepsis include shaking chills, fever, hypotension, oliguria, and confusion,

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit, according to the context provided on pages 2453-2454 under Chapter 227, includes the following steps:

1. Provide first aid: Keep the patient warm, control hemorrhage, check and secure the airway and ventilation, turn the patient's head to one side to avoid aspiration if emesis occurs, and do not give anything by mouth.
2. Begin treatment simultaneously with evaluation: Provide supplemental oxygen via face mask, intubate and mechanically ventilate if necessary, insert two large IV catheters into separate peripheral veins, or use a central venous line or intraosseous needle if prompt access to peripheral veins is not possible.
3. Administer aggressive fluid resuscitation to maintain adequate tissue perfusion.
4. Initiate antibiotic therapy as soon as possible based on culture and sensitivity results or clinical suspicion.
5. Perform surgical excision of infected or necrotic tissues and drain pus if needed.
6. Provide supportiv

Llama.generate: prefix-match hit


A: According to the context provided on pages 2453-2454 of The Merck Manual of Diagnosis & Therapy, 19th Edition, the protocol for managing sepsis in a critical care unit involves the following steps:

1. Patient Monitoring and Testing: Sepsis is a serious condition that requires close monitoring in an ICU setting. The patient's vital signs, including temperature, heart rate, respiratory rate, blood pressure, and oxygen saturation, should be closely monitored (p. 2453).
2. First Aid: The first step in managing sepsis is to provide basic first aid, which includes keeping the patient warm, controlling any bleeding, checking the airway and ventilation, and providing respiratory assistance if necessary (p. 2453).
3. General Management: In addition to first aid measures, the patient should receive supplemental oxygen via a face mask, and large IV catheters should be inserted into separate peripheral veins for fluid resuscitation. A central venous line or intraosseous needle may be used if p

### Grounded Question Execution

**Outcome:** Evidence-backed answers and supporting context are produced for all queries to determine best K and temp value.

**Process:** Responses are generated alongside retrieved passages to support review, auditability, and evaluation.

### Final RAG Answers (Best Configuration: k=3, temp=0.1)

In [ ]:
# ============================================================
# Final RAG Configuration -> Answers
# ============================================================

rag_responses = []
rag_contexts = []

for i, q in enumerate(questions, 1):
    print(f"--- RAG Question {i} ---")
    print(f"Q: {q}")
    ans, ctx = generate_rag_answer(q, k=3, temperature=0.1)
    print(f"A: {ans}\n")
    rag_responses.append(ans)
    rag_contexts.append(ctx)

--- RAG Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081, the common symptoms of pulmonary embolism include dyspnea, pleuritic chest pain, cough, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism involves anticoagulants, clot dissolution with thrombolytics, or surgical removal (Chapter 194, Introduction).

(Page 2080, Introduction)
"Symptoms are nonspecific and include dyspnea, pleuritic chest pain, cough, and, in severe cases, syncope or cardiorespiratory arrest. Signs are also nonspecific and may include tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound."

(Page 2080, Treatment)
"Treatment is with anticoagulants and, sometimes, clot dissolution with thrombolytics or surgical removal."

--- RAG Question 2 ---
Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?


Llama.generate: prefix-match hit


A: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which is then followed by a shift in pain to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left lower quadrant (Merck Manual, p. 174, Symptoms and Signs section).

Appendicitis cannot be cured via medicine alone as it requires surgical removal to prevent complications such as necrosis, gangrene, perforation, abscess formation, or inflammatory masses (Merck Manual, p. 174, Etiology section). The surgical procedure for treating appendicitis is typically an appendectomy, which may involve removing the appendix and draining any abscesses that have formed beforehand (Merck Manual, p. 1

...

A: Based on the information provided in The Merck Manual of Diagnosis & Therapy, 19th Edition, here are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip:

1. The person should be kept off their feet for several days to promote better healing (Page 3387). They may use crutches to help them move around.
2. Wound care is essential, keeping the wound clean and dry, applying nonadherent and impermeable bacteria-resistant dressings, and using antibiotic ointment daily until the wound closure device is removed (Page 3387).
3. For life- or limb-threatening injuries, hemorrhagic shock should be treated in the emergency department, and fractures to arteries may require surgical repair (Page 3391).
4. The person's injury should be immobilized immediately by splinting to prevent further damage to soft tissues (Page 3391).
5. Definitive treatment for certain injuries, such as reduction, may also be necessary (Page 3391).
6. Rest, ice, compr

Llama.generate: prefix-match hit


A: Based on the context provided, the diagnostic steps for suspected endocrine disorders include:

1. Measuring levels of peripheral endocrine hormones and pituitary hormones in the blood (Page 895 and 947).
2. Detecting autoantibodies to each affected glandular tissue to differentiate Primary Diagnosis from other causes (Page 947).
3. Elevated levels of pituitary tropic hormones suggest the hypothalamic-pituitary axis is intact, although some patients with type II PDS may have hypothalamic-pituitary insufficiency (Page 947).
4. Lifelong follow-up is prudent due to the potential for late manifestations of endocrine disorders, which can be life-threatening (Page 947).
5. Relatives should be made aware of the diagnosis and screened when appropriate; measurement of glutamic acid decarboxylase antibodies may be useful in determining risk (Page 947).

Therefore, the diagnostic steps involve a combination of laboratory testing to measure hormone levels and autoantibodies, as well as long-ter

Llama.generate: prefix-match hit


A: According to the context provided on pages 2453-2454 of "Critical Care Medicine," the protocol for managing sepsis in a critical care unit includes the following steps:

1. First aid: Keeping the patient warm, controlling hemorrhage, checking and securing the airway and ventilation, turning the patient's head to one side if emesis occurs, and providing nothing by mouth. (p. 2453)
2. Supplemental oxygen: Providing oxygen through a face mask. (p. 2453)
3. Airway intubation and mechanical ventilation: If the patient's shock is severe or if ventilation is inadequate, airway intubation with mechanical ventilation is necessary. (p. 2453)
4. Intravenous access: Inserting two large IV catheters into separate peripheral veins, and using a central venous line or an intraosseous needle if peripheral veins cannot be promptly accessed. (p. 2453)
5. Aggressive fluid resuscitation: Providing fluids to maintain adequate tissue perfusion. (p. 2453)
6. Antibiotics: Administering antibiotics to treat 

### RAG Analysis

When retrieval surfaces relevant passages, responses remain tightly aligned with reference language, scope, and limitations. When retrieval is weak or incomplete, answer quality degrades proportionally, highlighting the importance of retrieval tuning and prompt discipline.

Reason for selecting k=3, temp=0.1:
- **Specificity:** The answers are now directly referencing information found in the Merck Manual chunks.
- **Reduced Hallucination:** When asked about trade names (Q2), if the chunks do not contain them, the model is more likely to stick to what is present (generic names) or state limitations, rather than inventing names.
- **Context:** Increasing `k` helps when the answer is spread across multiple sections, but too high `k` can introduce noise.

## Automated Output Evaluation (LLM as the judge)

### Response Scoring

**Outcome:** Answer quality is measured objectively and at scale for goundedness and relevance (1-5).

**Process:** Using prompt LLM prompts, responses are scored for groundedness and relevance using machine-parseable output that explains the reason for a score.

In [ ]:
# ============================================================
# Response Evaluation (Refined 1-5 Scoring with evidence)
# ============================================================

import re

groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
5. IMPORTANT: End your response with a single line: SCORE: X (where X is 1, 2, 3, 4, or 5)
"""

relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
5. IMPORTANT: End your response with a single line: SCORE: X (where X is 1, 2, 3, 4, or 5)
"""

def evaluate_response(query, answer, context):
    # Groundedness Prompt
    ground_prompt = f"[INST] {groundedness_rater_system_message}\n\n###Question\n{query}\n###Context\n{context}\n###Answer\n{answer}\n[/INST]"

    # Use Main LLM (Mistral)
    ground_out = llm(ground_prompt, max_tokens=512, temperature=0.1)['choices'][0]['text'].strip()

    # Relevance Prompt
    rel_prompt = f"[INST] {relevance_rater_system_message}\n\n###Question\n{query}\n###Context\n{context}\n###Answer\n{answer}\n[/INST]"

    # Use Main LLM (Mistral)
    rel_out = llm(rel_prompt, max_tokens=512, temperature=0.1)['choices'][0]['text'].strip()

    # Parse Scores using Regex (Prioritize explicit format)
    def parse_score(text):
        # Try strict format first: SCORE: 5
        strict_matches = re.findall(r'SCORE:\s*([1-5])', text, re.IGNORECASE)
        if strict_matches:
            return int(strict_matches[-1])

        # Fallback: Look for last digit 1-5
        matches = re.findall(r'\b([1-5])\b', text)
        if matches:
            return int(matches[-1])
        return 0 # Error or not found

    ground_score = parse_score(ground_out)
    rel_score = parse_score(rel_out)

    return ground_score, ground_out, rel_score, rel_out

print("--- Evaluation Results ---\n")
for i, (q, a, c) in enumerate(zip(questions, rag_responses, rag_contexts), 1):
    gs, ge, rs, re_text = evaluate_response(q, a, c)
    print(f"Q{i}: {q}")
    print(f"Groundedness Score: {gs}/5")
    print(f"Reason: {ge}")
    print(f"Relevance Score: {rs}/5")
    print(f"Reason: {re_text}")
    print("-" * 50)


--- Evaluation Results ---



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q1: What are the common symptoms and treatments for pulmonary embolism?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the common symptoms of pulmonary embolism mentioned in the context.
2. Check if the AI generated answer includes those common symptoms.
3. Verify if the treatment options for pulmonary embolism as stated in the context are included in the AI generated answer.

The AI generated answer adheres to the metric as it derives the information only from the context provided. The answer mentions the common symptoms of pulmonary embolism (dyspnea, pleuritic chest pain, cough, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound) that are explicitly stated in the context. Additionally, it includes the treatment options for pulmonary embolism (anticoagulants, clot dissolution with thrombolytics, or surgical removal) as mentioned in the context.

Evaluation: The metric is followed completely.
SCORE: 5
Relevance Score: 4

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the common symptoms of appendicitis mentioned in the context.
2. Check if the AI generated answer includes those common symptoms.
3. Determine if the AI generated answer mentions that appendicitis cannot be cured via medicine and requires surgical removal.
4. Verify if the surgical procedure for treating appendicitis, as stated in the answer, is consistent with what's mentioned in the context.
5. Ensure that all information in the answer is derived solely from the context.

The AI generated answer adheres to the metric as it includes the common symptoms of appendicitis and correctly states that appendicitis cannot be cured via medicine and requires surgical removal, which are all derived directly from the context.

Evaluation: The metric is followed completely.

SCOR

...

Q7: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the necessary precautions and treatment steps for a person who has fractured their leg based on the context.
2. Compare the identified steps with the AI generated answer to ensure they match.
3. Evaluate if the answer is derived only from the information presented in the context.

The answer adheres to the metric as it provides the necessary precautions and treatment steps for a person who has fractured their leg based on the information provided in the context of The Merck Manual of Diagnosis & Therapy, 19th Edition.

Extent to which the metric is followed: 5 - The metric is followed completely.

SCORE: 5
Relevance Score: 4/5
Reason: Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the questio

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q8: What are the diagnostic steps for suspected endocrine disorders?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the diagnostic steps mentioned in the context.
2. Compare the steps listed in the context with those in the AI generated answer.
3. Determine if the AI generated answer is derived only from the information presented in the context.

The AI generated answer adheres to the metric as it lists the exact diagnostic steps mentioned in the context. Therefore, the extent to which the metric is followed is 5 - The metric is followed completely.

SCORE: 5
Relevance Score: 4/5
Reason: Steps to evaluate the context as per the metric:
1. Identify the main aspects of the question related to diagnostic steps for suspected endocrine disorders.
2. Determine if the context provides sufficient information on these aspects.
3. Evaluate how well the context explains each step in detail and relevance to the question.

The context adheres to the metric considering the

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q9: What is the protocol for managing sepsis in a critical care unit?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the main points of the context related to managing sepsis in a critical care unit.
2. Check if the AI generated answer includes all or most of these main points.
3. Determine if any additional information is included in the answer that is not present in the context.
4. Evaluate if the answer is derived solely from the context and not from external sources.

The AI generated answer adheres to the metric as it includes all the necessary steps for managing sepsis in a critical care unit as outlined in the context. The answer does not include any additional information that is not present in the context. Therefore, the metric is followed completely.

SCORE: 5
Relevance Score: 4/5
Reason: Steps to evaluate the context as per the metric:
1. Identify the main aspects of the question related to managing sepsis in a critical care unit.
2. Determine if t

##Summary: Business value and deployment strategy defined

### Next Steps: Model and Data Enhancements

**Broaden the reference corpus:** Incorporate additional peer-reviewed medical literature and specialty-specific guidelines to expand clinical coverage and reduce dependency on a single source.

**Improve retrieval fidelity**:
Further optimize chunking strategy, embedding selection, and top-k retrieval parameters to ensure critical clinical context is consistently surfaced, particularly for complex or cross-sectional queries.

**Adopt domain-specialized embeddings**:
Evaluate medical-domain embedding models to improve semantic accuracy for nuanced clinical terminology.

**Strengthen evaluation rigor**:
Expand automated and human-in-the-loop evaluation using real-world clinical queries to monitor performance over time and identify degradation or bias.

**Productization and System Readiness**:
evelop a front-end dashboard
Implement a secure, user-facing interface that enables clinicians and reviewers to:

**Submit queries**:
View generated answers alongside cited source excerpts

**Inspect relevance and groundedness scores**:
This capability is essential for transparency, trust, and executive oversight.

**Enable auditability and governance**:
Persist query logs, retrieved references, and model outputs to support compliance, internal review, and regulatory readiness.

**Optimize deployment architecture**:
Assess inference performance, cost, and scalability trade-offs to support broader adoption across clinical environments.

**Strategic and Business Considerations**:
Pilot deployment
Introduce the system to a limited cohort of healthcare professionals to validate usability, workflow impact, and time savings.

**Clinical domain expansion**:
Extend the solution into high-impact specialties (e.g., emergency medicine, critical care) where rapid access to grounded information is especially valuable.

**Clear positioning and risk management**:
Maintain explicit positioning as a clinical decision-support tool, reinforcing that it augments—not replaces—professional medical judgment.

### Expanded Executive Summary

Healthcare organizations increasingly seek to leverage AI to improve efficiency and decision-making; however, the use of unconstrained generative models in clinical settings introduces unacceptable risk. This project directly addresses that concern by implementing a system architecture that prioritizes traceability, evidentiary grounding, and professional rigor. Rather than generating responses solely from a model’s internal knowledge, the system retrieves relevant passages from a trusted medical reference and restricts generation to that retrieved content.

From a technical perspective, the system processes more than 4,000 pages of structured medical content, converts them into semantically indexed representations, and retrieves the most relevant sections for each query. These sections are incorporated into carefully controlled prompts that enforce a professional medical tone and explicit citation behavior. This design significantly reduces hallucination risk and ensures that outputs remain aligned with established clinical guidance.

The system’s value is already evident. Automated evaluations consistently confirm that responses are both relevant to the question posed and grounded in source material. This translates into practical benefits for healthcare professionals, including reduced manual search effort, more standardized diagnostic information, and increased confidence in AI-assisted insights.

Looking ahead, the platform provides a strong foundation for scalable clinical decision support. By expanding reference sources, improving retrieval precision, and introducing a transparent front-end interface, the system can evolve into a production-grade solution aligned with organizational governance requirements. Importantly, the architecture supports a balanced approach to innovation—enabling leadership to capture the benefits of AI while maintaining control over accuracy, risk, and accountability.